In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime


# Load .env
load_dotenv()

True

In [ ]:
engine = create_engine(
    f"postgresql+psycopg://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

: 

In [ ]:
df = pd.read_sql("SELECT * FROM customers", engine)
df.head()

In [ ]:
df.info()
df.describe()
df['churned'].value_counts(normalize=True)

In [ ]:
df.corr(numeric_only=True)
df.groupby('churned').mean(numeric_only=True)

In [ ]:
sns.boxplot(x='churned', y='monthly_spend', data=df)
plt.title("Monthly Spend vs Churn")
plt.show()

In [ ]:
df["signup_date"] = pd.to_datetime(df["signup_date"])
df["last_active_date"] = pd.to_datetime(df["last_active_date"])
today = pd.to_datetime("today")

df["tenure_days"] = (today - df["signup_date"]).dt.days

df["logins_per_day"] = df["logins_last_30_days"] / 30

df["tickets_per_month"] = df["support_tickets"] / (df["tenure_days"] / 30 + 0.01)

In [ ]:
features_for_model = [
    "monthly_spend",
    "tenure_days",
    "logins_per_day",
    "tickets_per_month",
    "support_tickets",
    "logins_last_30_days"
]

X = df[features_for_model] 
y = df['churned']          

model_data = pd.concat([X, y], axis=1)  
model_data.to_csv("../data/processed/customers_clean.csv", index=False)

print("Feature Correlations:")
print(X.corr())

plt.figure(figsize=(10, 6))
sns.heatmap(X.corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/customers_clean.csv")

print("Duplicates:", df.duplicated().sum())

print("Missing values:\n", df.isnull().sum())

print("Churned value counts:\n", df['churned'].value_counts(normalize=True))

print("Correlation with churned:\n", df.corr(numeric_only=True)['churned'].sort_values(ascending=False))

df = pd.read_csv("../data/processed/customers_clean.csv")

for col in ["logins_per_day", "tickets_per_month"]:
    print(f"\n{col} by churned status:")
    print(df.groupby("churned")[col].describe())